# Introduction to Neuroinformatiocs
## Exercise session 4: Active membrane properties

Welcome to the fourth exercise of the introduction to neuroinformatics course. In the previous exercise, we introduced the **Integrate-and-No-Fire** neuron model, focusing on how synaptic inputs allow currents to alter the membrane potential of a neuron. We explored how these <ins>inputs are integrated and summed at the soma</ins>, determining the overall effect on the neuron's membrane potential. Additionally, we briefly touched on the concept of the **action potential**, a <ins>non-linear, all-or-nothing response</ins> triggered when the <ins>membrane potential crosses a specific threshold</ins>. This **"spike"** represents a critical event in neural communication, as it enables the neuron to send signals to downstream targets.

In the following exercise we will see that different types of inputs can have different effects on the postsynaptic membrane potential. Further, we will intruduce the Integrate-and-Fire neuron, a model that can produce **spikes**.

# Table of contents

* [Packages](#packages)
* [1: Synapses](#1)
    * [1.1: Synaptic input](#1.1)
    * [1.2: Equivalent circuit of a synapse](#1.2)
* [2: Input integration and the Integrate-and-Fire model](#2)
    * [2.1: Temporal EPSP integration](#2.1)
    * [2.2 The Hodgekin-Huxley model](#2.2)
    * [2.3 The Integrate-and-Fire model](#2.3)
    * [2.4: The Leaky-Integrate-and-Fire Neuron model](#2.4)

# Packages <a class="anchor" id="packages"></a>

This notebook was <font color='red'>**designed to be run in colab**</font>. We have included code blocks to allow the possibility of running the notebook locally, but due to the vastness of peoples' setups, we do not support it much further.

If you wish to continue in Colab, click on the "Copy to Drive" button to be able to save your progress.

The following cells are used to install the necessary packages and libraries for the exercise:

In [1]:
!which python  # This displays which python is being used
!pwd # This displays the current directory

/usr/local/bin/python
/content


**IMPORTANT**:

 Interactive plots act strange in google colab, so after after running the following cell, restart your session (in the dropdown under "Run all").

In [2]:
!pip3 install --quiet ipywidgets ipympl wget # Install necessary packages

In [3]:
platform = 'notebook'
try:
    import google.colab
    from google.colab import output
    platform = 'colab'
    output.enable_custom_widget_manager()
except:
    pass
print(f"platform: {platform}")

platform: colab


In [4]:
if platform == 'colab':
    %matplotlib widget
else:
    %matplotlib inline


In [15]:
import wget
import os

# Define the URL and the output directory
url = 'https://raw.githubusercontent.com/ManteLab/Iton_notebooks_public/refs/heads/main/utils_ex4/utils.py'
output_dir = 'utils_ex4'

if not os.path.exists(output_dir):
    # Create the directory if it doesn't already exist
    os.makedirs(output_dir, exist_ok=True)

    # Download the file into the specified directory
    print(f"Downloading file to '{output_dir}'...")
    filename = wget.download(url, out=output_dir)
    print(f"\nFile successfully downloaded as: {filename}")


# 1. Synapses <a class="anchor" id="1"></a>

## 1.1: Synaptic input <a class="anchor" id="1.1"></a>



Recall the following **key concepts**:

**Depolarization**: Neurons <ins>at rest</ins> have a <ins>membrane potential</ins> of approximately <ins>-70 mV</ins>, which we refer to as a <ins>polarized</ins> state (i.e., a state far from 0 mV). **Depolarizing currents** bring the membrane potential <ins>closer to 0 mV</ins>.

**Hyperpolarization**: In some cases, synaptic input results in a **hyperpolarizing effect**, driving the membrane potential even further away from 0 mV, thus making the neuron less likely to fire.

**EPSP** (**E**xcitatory **P**ost**s**ynaptic **P**otential): Synaptic input that has a depolarizing effect, moving the membrane potential closer to the threshold for an action potential.

**IPSP** (**I**nhibitory **P**ost**s**ynaptic **P**otential): Synaptic input that causes hyperpolarization, moving the membrane potential further from the firing threshold.

Whether a <ins>synaptic input is excitatory or inhibitory</ins> depends on the **neurotransmitter** (**NT**) released by the presynaptic neuron. According to **Dale's Principle**, a neuron releases the <ins>same neurotransmitter at all of its synapses throughout its lifetime</ins>, maintaining a consistent chemical action.

In the brain, the <ins>primary excitatory neurotransmitter</ins> is **glutamate**, while the <ins>main inhibitory neurotransmitter</ins> is **GABA** (gamma-Aminobutyric acid).

Note that so far all synaptic inputs in our models have been excitatory.

## 1.2 Equivalent circuit of a synapse <a class="anchor" id="1.2"></a>

The figure below depicts a simplified neuron, represented by an equivalent circuit including a fast chemical synapse. $V_{rest}$ is the resting potential of the cell (here denoted as the **reversal potential** of the **leakage current** $E_L$), $R_L$ is the input resistance and $V_m(t)$ is the membrane potential dependent on time $t$.

---

Note, that <ins>synaptic input</ins> occurs only if <ins>specific ion channels</ins> in the postsynaptic membrane <ins>open</ins>. These channels are often <ins>selectively permeable for multiple ion types</ins> and in order for them to open, they have to bind <ins>presynaptically released NT</ins>.

---

In the example below, $E_{syn}$ is the equilibrium potential of the synapse and $g_{syn}$ is the synaptic conductance. Following Ohm's law you obtain the synaptic current $I_{syn}$ (generated by a single synapse depicted on the left side of the diagram by:

$I_{syn} = g_{syn}(t) (V_m(t) - E_{syn})$


Note that the synaptic conductance $g_{syn}$ is a function of time and depends on the opening of synaptic ion channels. The complete equation for the above circuit (also implementing Kirchhoff's law) yields:

$0 = I_{syn} + I_L = g_{syn}(t) (V_m(t) - E_{syn}) + (V_m(t) - V_{rest}) / R_L$


Assume that the synapse is "switched on" at t = 0 (after a presynaptic action
potential arrives at the synapse and NT binds to the postsynaptic channels), and current may pass through. Further, $g_{syn}$ is "switched off" again at t = 1 (when all NT has been cleared again from the synaptic cleft):

$g_{syn}(t) =
\begin{cases}
0 & \text{for } t < 0, \\
1 nS & \text{for } 0 < t \leq 1, \\
0 & \text{for } t > 1
\end{cases}$

Assume $V_{rest} = 70\text{mV}$ and $R_L = 2G\Omega$

![My Image](https://github.com/ManteLab/Iton_notebooks_public/blob/main/utils_ex4/ex4_synapse_circuit.png?raw=true)

---

> **Assignment 1**
>
> Let $E_{syn} = 10\text{mV}$. Is this an excitatory or inhibitory synapse? Calculate $I_{syn}$ and the amplitude of the postsynaptic potential (i.e. $V_m(t)$ for $0\text{ms} < t < 1\text{ms}$).

<font color='red'>**Solution:</font>**

$0 = I_{syn} + I_L = g_{syn}(t) (V_m(t) - E_{syn}) + (V_m(t) - V_{rest}) / R_L \Leftrightarrow V_m(t) = \frac{g_{syn}(t){E_{syn}+V_{rest}/R_L}}{g_{syn}(t)+1/R_L}$

$V_m(t) = -16.67\text{mV}$

The synapse is excitatory.

$I_{syn} = g_{syn}(V_m(t)-E_{syn})$

$I_{syn} = -26.67\text{pA}$

---



> **Assignment 2**
>
> Let $E_{syn} = -90\text{mV}$. Is this an excitatory or inhibitory synapse? Calculate $I_{syn}$ and the amplitude of the postsynaptic potential.

<font color='red'>**Solution:</font>**

$V_m(t) = \frac{g_{syn}(t){E_{syn}+V_{rest}/R_L}}{g_{syn}(t)+1/R_L}$

$V_m(t) = -83.33\text{mV}$

The synapse is inhibitory.

$I_{syn} = g_{syn}(V_m(t)-E_{syn})$

$I_{syn} = 6.67\text{pA}$

---



> **Assignment 3**
>
> Let $E_{syn} = -70\text{mV}$. Calculate $I_{syn}$ and the amplitude of the postsynaptic potential. Calculate $g_{total}$ (total membrane conductance). What effect will this synapse have on the responsiveness of the membrane? Is this an excitatory or inhibitory synapse?

<font color='red'>**Solution:</font>**

$V_m(t) = -70\text{mV}$

$I_{syn} = 0\text{pA}$

$g_{total} = g_{syn} + 1/R_L$

Due to additional conductances, the synapse will be less responsive to excitatory current. This form of inhibition is called **shunting inhibition**.

---

# 2. Input integration and the Integrate-and-Fire model <a class="anchor" id="2"></a>



## 2.1: Temporal EPSP integration <a class="anchor" id="2.1"></a>

In [17]:
from utils_ex4.utils import iplot_InoF_model
# Integrate and No fire
iplot_InoF_model()

The **interactive plot** above allows you to simulate input to a neuron with a single synapse.  *Note that the new design of the interactive plot requires you to click "**Update Plot**", after you changed parameters. This should allow the code in the background to run faster.*

Depending on how many action potentials the presynaptic cell fires (Input Frequency), more or fewer EPSPs accumulate at the soma. Change the parameters to get and feeling for their effect on the membrane potential in this model and attend to the **assignments** below.

---

> **Assignment 4**
>
> Change the parameter "Distance: > Synapse-Soma" while keeping other parameters unchanged. Then, change the "Input Frequency" while keeping other parameters unchanged. Explain why these changes introduce different delays in the occurance of EPSPs.

<font color='red'>**Solution:</font>**

Frequency effects the distance between two subsequent EPSPs. The higher the frequency, the shorter the interval. It does not influence the time until the first input is measurable at the soma.
On the other hand, the distance from the synapse to the soma directly influences the delay. Since this delay is the same for all inputs, it does not change the interval between EPSPs.

---



> **Assignment 5**
>
> Does the model incorporate the membrane time constant $\tau$ or the  length constant $\lambda$? Explain how you came to your conclusion.

<font color='red'>**Solution:</font>**

No, both constants are not implemented in this model.

$\tau$ would affect the membrane potential depolarization so that the voltage would drop over time (along the x axis).
$\lambda$ would attenuate the membrane potential depolarization relative to the distance of the input to the soma (the further the input travels along the dendrite, the more it attenuates.)

---



----



## 2.2: The Hodgkin-Huxley model <a class="anchor" id="2.2"></a>

The **model presented in Section 2.a** allows for indefinite depolarization of a neuron by linearly summing all EPSPs arriving at the soma. However, as we have seen, this behavior does not at all represent how real neurons function. When the membrane potential crosses a certain **depolarization threshold**, neurons generate a <ins>non-linear, all-or-nothing response</ins>: **The action potential**.

In 1952, Alan Hodgkin and Andrew Huxley developed a <ins>mathematical model</ins> that explains how action potentials are <ins>initiated and propagated</ins> along the squid giant axon.

$I_{\text{ion}} =
\underbrace{g_{L} (V_{m} - E_{L})}_{\text{Leak current, }I_{L}} +
\underbrace{\overline{g_{K}} *  n^4 (V{m} - E_{K})}_{\text{Potassium current, }I_{K}} +
\underbrace{\overline{g_{Na}} * m^3 * h (V_{m} - E_{Na})}_{\text{Sodium current, }I_{Na}}
$

The **total ionic current** $I_{ion}$ is a <ins>sum</ins> of the <ins>constant currents</ins> through so called **leak channels**, $I_{L}$ and the <ins>currents that are responsible for fast de- and repolarisation during an action potential</ins>: $I_{Na}$ and $I_{K}$. As we have seen in previous exercises, the current for each ion depends on **a)** the selective permeability of the membrane for the ion species (conducatence, $g_{ion}$), and **b)** the difference between the membrane potential and the reversal potential of the ion species:

$I_{ion} = g_{ion} * (Vm - E_{ion})$

For several ions, the conductance $g_{ion}$ is not constant but modulated by a physical gate that is opened or closed with a certain probability. The probability for one of the ion channels to be open is given by $n_{ion}$. Thus, $n_{ion}$ can be interprated as a proportion of the maximum conductance.

The actual ionic conductance can therefore be written as the maximal conductance $\overline{g_{ion}}$ multiplied by the gating variable so that:

$g_{ion}$= $\overline{g_{ion}}$  * $n_{ion}$

With this is mind, attend to the following **assignments**:

---

> **Assignment 6**
>
> Answer the following questions when interpreting the Hodgkin-Huxley model:
> - What are the lower and upper bound for $n_{ion}$?
> - Suppose that each channel has several gates in series (let's say $x$ gates), all of which have to be open to permit ions to pass through the channel. If the probability of a single gate to be open is $n$, what is the probability that all $x$ gates are open?
> - Each gate can change from an open to a closed state and vice-versa. If the probability that a gate in a closed state will open is $\alpha_n$, and the probability that an open gate closes is $\beta_n$, how does the
> proportion of open gates $n$ change? $dn / dt =$ . . .
>  - In steady state, $n$ does not change anymore: $dn / dt = 0$. What is the gating variable $n$ in terms of $\alpha_n$and $\beta_n$ at steady? $n_{∞} =$ . . .
> - Hodgkin & Huxley found that a model with 4 gates in series produces a good fit to the S-shaped curve of the potassium current ($I_{K}$) during the action potential. Explain the second term ($I_{K}$) of their model.
> - In the third term ($I_{Na}$) Hodgkin & Huxley introduced another variable termed $h$. It reflects a specific property of the sodium channels involved in action potential generation: Inactivation. Once, these sodium channels have opened and closed again, they will resist to open again for a certain amount of time. Explain the third term ($I_{Na}$) of their model.

<font color='red'>**Solution:</font>**

- Probability: $0 ≤ n_{ion} ≤ 1$; $n_{ion}=0$: all channels are cloded;  $n_{ion}=1$: all channels are open
- Probability for $1$ gate open is $n$, probability for $2$ gates open is $n*n = n^2$, probability for $x$ gates open is $n_1*n_2*…*n_x = n^x$
- Open gates: $n$, closed gates: $1-n$; $\frac{dn}{dt}=α_n*(1-n) - β_n*n$
- At steady-state: $\frac{dn_∞}{dt}=α_n*(1-n) - β_n*n = 0$
  
  $n_∞=\frac{α_n}{α_n+β_n}$

- $I_K = \overline{g_{K}} *  n^4 (V{m} - E_{K})$

  $\overline{g_{K}}$: max possible conductance

  $n$: probability that one of 4 gates is open

  $n^4$: probability that whole channel is open ($g_K$ is modelled with 4 gates in series)

  $V_m – E_K$: Electrochemical potential of potassium



- $I_{Na} = \overline{g_{Na}} * m^3 * h (V_{m} - E_{Na})$

  $\overline{g_{Na}}$: max possible conductance

  $m$: probability that one of 3 gates is open

  $n^3$: probability that whole channel is open ($g_{Na}$ is modelled with 3 gates in series)

  $V_m – E_{Na}$: Electrochemical potential of sodiun
  
  $h$: probability that an additional, independent gate inactivates the channel. Note: If $h=0$, the states of n have no effect.

---

## 2.3: The Integrate-and-Fire neuron model <a class="anchor" id="2.3"></a>

In <ins>contrast to the Integrate-and-no-Fire model</ins>, the **Integrate-and-Fire neuron model** triggers a **spike** (action potential) as soon as threshold is reached and resets the membrane potential to resting potential immediately afterwards.

In the **interactive plot** below we extend the previous model (section 2.a) to a spiking (IF neuron) model. The second plot below is identical to the first plot in section 2.a, except that we now added an adjustable parameter: the action potential threshold. The first plot shows the resulting spike train resulting from the depolarization levels in plot 2.

Feel free to change parameters of the model and see what happens.

In [18]:
from utils_ex4.utils import iplot_Integrate_and_Fire_model

In [19]:
# Integrate and Fire model
iplot_Integrate_and_Fire_model()

In the **interactive plot** above, set the <ins>weight to maximum (100)</ins>, the <ins>distance to it's minimum value (0.1mm)</ins>, the <ins>AP threshold ot 20mV</ins> and the <ins>refractory scale factor to 0.5</ins>. Compare the spike trains between <ins>input frequencies of 220-230</ins> to input frequencies of <ins>320-330</ins>. Do another comparison with the <ins>refractory scale factor to 0.2<\ins>.



---

> **Assignment 7**
>
> How do the resulting spike trains differ? What is the function of the refractory scale factor?

<font color='red'>**Solution:</font>**

This model implements the refractory scale factor is a function of the level of depolarization at the time a spike is triggered. At high frequencies, the membrane potential after spike 1 rises faster. When the first refractory period is over, higher input frequencies lead to a very long refractory period after the second spike.

---




---

> **Assignment 8**
>
> When looking at the resulting plot, try to relate it's dynamics to the mathematical model of Hodgkin and Huxley from section 2.b. Link the terms in their formular to observations in the interactive model.

<font color='red'>**Solution:</font>**

The observations in the plot are only connected vaguely to the Hodgkin-Huxley model. However, the connections that can be made are striking. The fast rise time of action potentials is linked to conductances of voltage-gated sodium channels and therefore related to $g_{Na}$ and $m^3$. Similar is the relation to voltage gated potassium channels and the term $I_K$. One additional connection can be draw between the refractory period and the term $h$ in $I_{Na}$ since this deactivation of voltage gated sodium channels is specifically what causes the refractory period.

## 2.4: The Leaky-Integrate-and-Fire Neuron model <a class="anchor" id="2.4"></a>

Slowly but surely our neuron model emulates more and more properties of a real neuron.

We have seen that, given a depolarizing current pulse at time $t$ and location $x$, the resulting EPSP attenuates over time and while propagating along a dendrite (**cable equation)**. This is due to a subset of **potassium channels** that are considered "leaky". These channels have a rather high probability of beeing open and are <ins>different to the repolarizing voltage-gated potassium channels</ins> found in axons involved in action potential generation. You have seen leak current ($I_{L}$) already as a part of the equivalent circuit of a synapse in section **1.b** of this exercise.

The model in the **interactive plot** below is a **Leaky-Integrate-and-Fire (LIF)** neuron - synaptic inputs now generate EPSPs that attenuate over time. Go ahead, change some of the parameters to get a feeling for the model and attend to the **assignments** below.

In [20]:
from utils_ex4.utils import iplot_Leaky_Integrate_and_Fire_model

In [21]:
# Leaky Integrate and Fire
iplot_Leaky_Integrate_and_Fire_model()

Since you now got a feeling on how the model behaves, try to find settings to generate one EPSP with an amplitude approximately 60% of the amplitude necessary to reach the spiking treshhold. Then, independenlty change the distance and the time constant parameters.

---

> **Assignment 9**
>
> Does this model implement $\tau$? If so, how can you tell?

<font color='red'>**Solution:</font>**

The time constant $\tau$ is indeed implemented. A generated EPSP doesn't stay at its initial maximum amplitude but decays over time.

---



Next, change the parameters to produce two or three EPSPs but keep the time constant at it's minimum value (1ms). Then slowly increase the time constant.


---

> **Assignment 10**
>
> How does this influence the neuron’s ability to integrate information?

<font color='red'>**Solution:</font>**

A larger time constant reduces the decay of an EPSP ($\tau$ is the time after which the EPSP amplitude decreases to ~36% of its maximum amplitude.). With large $\tau$, EPSPs haven't attenuated before the next EPSP arrives, thus allowing the neuron to perform linear summation on the EPSPs.

---


Next, set the following parameters: $\text{Input frequency} = 10\text{Hz}$, $\text{weight} = 50\text{mV}$, $\text{distance} = 1\text{mm}$, $\text{theshold} = 100\text{mV}$ and $\text{time constant} = 50\text{ms}$.


---


> **Assignment 11**
>
> Calulate the membrane potential at $t=60\text{ms}$, then change the input frequency to 20\text{Hz} and calulate the membrane potential at $t=60\text{ms}$ again.

<font color='red'>**Solution:</font>**

Based on what we learned from cable theory:

- For input = 10 Hz:

 $$V_m(t) = V_m * (1 - e^{-t / τ}) \Leftrightarrow V_m(t=\tau) = V_m * (1 - e^{-1}) = 0.63 * V \text{(for time $t = \tau$)}$$

  $$\implies V_m(60\text{ms}) = -51.605 \text{mV}$$

- For input = 20 Hz:

  Second spike at $50\text{ms}$ reaches the soma at $60 \text{ms}$

  From previous calc. $V_m(60\text{ms}) = -51.605 \text{mV}$ <br>
  Therefore the current spike adds on to this value, i.e. $V_m(60\text{ms}) = -51.605 \text{mV} + 50 \text{mV} = -1.605 \text{mV}$


---


At last, set the following parameters: $\text{Input frequency} = 10\text{Hz}$, $\text{weight} = 30\text{mV}$, $\text{distance to soma} = 0.1\text{mm}$, $\text{theshold} = 20\text{mV}$, $\text{time constant} = 30\text{ms}$ and $\text{refractory scale factor} = 0.01$.



---

> **Assignment 12**
>
> Increase the distance to soma to 1mm and then to 2mm. What changes? When comparing your observations with the results you expect from cable theory, which parameter is not implemented in this model? How would the results change, if this parameter was implemented?  


<font color='red'>**Solution:</font>**

The EPSP does not change in amplitude or shape. If all parameters from cable theory were implemented, the length constant ($λ$) would lead to an attenuation of the EPSP amplitude with increasing distance of the synapse to the soma. Thus, the maximum EPSP amplitude would become smaller.

---

Up to this point, all of our plots in this exercise have modeled a neuron receiving input from only a single synapse. However, in biological brains and more complex artificial neural networks, individual neurons typically receive inputs from dozens, hundreds, or even over a thousand synapses.

The interactive plot below allows you to model a neuron with **20 synapses**. In <ins>previous examples</ins>, you were able to <ins>modify parameters for a single synapse</ins>, such as its distance from the soma or its synaptic strength (weight). As you can imagine, manually setting these parameters for 20 synapses would be a time-consuming task. To simplify this, the <ins>sliders below</ins> allow you to <ins>specify **distributions**</ins> (mean and standard deviation) for these values. From these distributions, 20 random values are drawn and assigned to the 20 synapses.

In [23]:

from utils_ex4.utils import iplot_Leaky_Integrate_and_Fire_with_distributions

In [24]:
iplot_Leaky_Integrate_and_Fire_with_distributions()

In order to understand the power of signal integration in single neurons, we will briefly introduce the concept of coding (precisely rate and temporal coding) here. You will hear more about these concepts during the lecture, so for know it is sufficient to keep it simple.

It is in general a difficult task to assign meaning to the spiking dynamics of neurons in the brain. What does the neuron encode when it fires so and so often or at these specific times? Two common coding strategies are well described and here we introduce them briefly:

**Rate coding**: The spiking frequency of a neuron is informative about some property of the signals. For example, a sensory neuron will fire action potentials at higher frequencies, when the input stimulus is stronger.

**Temporal coding**: The firing frequency itself is not informative about the encoded information but rather temporal factors like the relative timing of single spikes.

With this in mind, set the following parameters for the neuron model above: $\text{Distance} = 1\text{mm} \pm 0.01$, $\text{Weight} = 5\text{mV} \pm 0.01$, $ \text{Time constant} = 10\text{ms}$ and $\text{Threshold} = 0\text{mV}$. Then find a set of parameters that maximises the amount of inputs while the neuron fires as few spikes as possible.





---

> **Assignment 13**
>
> Which parameters did you find? Explain their influence.

<font color='red'>**Solution:</font>**

In general, high input rates increase the number of total inputs the most but also lead to early and frequent spikes. To sustain a sub-threshold regime, the input frequency can be high if the synchronisation is at its minimum.

---




Now change the parameters to provoke some more action potentials (2-3 spikes are sufficient) in the neuron. Individually change the parameter distributions and observe the neuron response in all your chosen parameter regimes.

---

> **Assignment 14**
>
> Does any of the distributions influence the neuron's output more than the others? Is there a particular distribution that critically influences the neuron's response so that - if not above / below a certain threshold - the neuron won't fire?

<font color='red'>**Solution:</font>**

It seems, that synchronisation of inputs has a strong influence on spike generation. However, even with low synchronisation, the neuron can be pushed to spike if e.g. the input weights or the input frequency are increased. A lower spike threshold also leads to more output. Thus, no single parameter on its own dictates the neurons output.

---



> **Assignment 15**
>
> Find parameter regimes so that the neuron implements rate coding.

<font color='red'>**Solution:</font>**

To be fair, this is a trick question when considering only a single neuron. Any regime, in which the output frequency reflects the input strenght could be considered to be rate coding.

---



> **Assignment 16**
>
> Find parameter regimes so that the neuron implements temporal coding.



<font color='red'>**Solution:</font>**

Same as in assignment 15, but even more pronounced. Setting the parameters so that the neuron fires a single spike at a specific time would implement a temporal code (time to first spike). Of course, this makes more sense in the context of a neural network. For e.g. 1 spike at a specific time, try high weights and strong synchronisation.

---


Set the following parameters: $\text{Weight} = 2.5\text{mV} \pm 2.5$, $ \text{Time constant} = 10\text{ms}$ and $\text{Threshold} = -20\text{mV}$.



---

> **Assignment 17**
>
> For different parameter regimes (e.g. high input rate with poor syncronsisation), how many inputs are required to provoke an action potential at $t=20\text{ms}$?



<font color='red'>**Solution:</font>**

Setting the distance to 2mm and using rather weak synapses, you will find that high synchronicity triggers action potentials with comparable few inputs. When decreasing the synchronicity, the input rates must increase drastically in order to trigger an action potential at the given time.

---

# End of this exercise session